In [1]:
import pathlib
from fytok.Tokamak import Tokamak
from fytok.modules.Equilibrium import Equilibrium
from fytok.utils.logger import logger
from spdm.view.View import display
from spdm.data.File import File
from spdm.data.Path import Path
from spdm.data.Entry import Entry

WORKSPACE = "/home/salmon/workspace"  # "/ssd01/salmon_work/workspace/"
output_path = pathlib.Path(f"{WORKSPACE}/output/")

2023-12-07 10:25:30,492 [    spdm]     INFO: 
###################################################################################################

    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 version = alpha  (spdm=beta extension=0.2.2.dev19+g14adade.d20231205, imas_version=v3_38_1_dirty)

 Run by salmon at 2023-12-07T10:25:30.171665.
 Job ID: fytok_salmon_surface_6942

###################################################################################################



In [2]:
# eq0 = Equilibrium(f"file+GEQdsk://{WORKSPACE}/gacode/neo/tools/input/profile_data/g141459.03890#equilibrium")
tok = Tokamak(
    "file+geqdsk:///home/salmon/workspace/fytok_tutorial/tutorial/data/g900003.00230_ITER_15MA_eqdsk16HR.txt",
    device="east",
    equilibrium={"code": {"name": "freegs"}},
)

2023-12-07 10:25:30,709 [    spdm]     INFO: Load module spdm.plugins.data.plugin_xml
2023-12-07 10:25:30,713 [    spdm]     INFO: Load module spdm.plugins.data.plugin_geqdsk


In [3]:
with File("test.geqdsk", mode="w", format="GEQDSK") as fid:
    fid.write({"equilibrium": {"time_slice": [tok.equilibrium.time_slice[0]]}, "wall": tok.wall})

2023-12-07 10:25:31,085 [    spdm]     INFO: Load module fytok.plugins.equilibrium.freegs
2023-12-07 10:25:31,089 [    spdm]     INFO: Initialize module fytok.modules.Wall 
2023-12-07 10:25:31,095 [    spdm]     INFO: Initialize module fytok.modules.TF 
2023-12-07 10:25:31,100 [    spdm]     INFO: Initialize module fytok.modules.Magnetics 
2023-12-07 10:25:31,106 [    spdm]     INFO: Initialize module fytok.modules.PFActive 
2023-12-07 10:25:31,109 [    spdm]     INFO: Initialize module fytok.plugins.equilibrium.freegs 
2023-12-07 10:25:31,128 [    spdm]     INFO: Load module spdm.mesh.mesh_rectangular


In [4]:
Entry(tok.equilibrium.time_slice[0]).get("boundary/geometric_axis/r")

6.201716537311407

In [5]:
tok.refresh()

2023-12-07 10:25:32,715 [    spdm]    DEBUG: /home/salmon/workspace/fytok/python/fytok/plugins/equilibrium/freegs.py:51:_setup_machine: Setup machine
2023-12-07 10:25:32,915 [    spdm]     INFO: Setup machine description: wall=<_T_identifier_static />
2023-12-07 10:25:32,916 [    spdm]    DEBUG: /home/salmon/workspace/fytok/python/fytok/plugins/equilibrium/freegs.py:93:preprocess: Setup freegs solver
2023-12-07 10:25:34,520 [    spdm]     INFO: Using free boundary
2023-12-07 10:25:34,523 [    spdm]    DEBUG: /home/salmon/workspace/fytok/python/fytok/plugins/equilibrium/freegs.py:132:preprocess: Setup profiles for equilibrium solver


RuntimeError: Can not create freegs profile!

In [ ]:
profiles_2d.grid